In [24]:
# 라이브러리 불러오기
'''메인 라이브러리'''
import numpy as np
import pandas as pd
import os, time, re
import pickle, gzip, datetime
from datetime import datetime

'''시각화 관련 라이브러리'''
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl

%matplotlib inline

'''데이터 준비 및 모델 평가 관련 라이브러리'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, mean_squared_error

'''알고리즘 관련 라이브러리'''
import lightgbm as lgb

'''텐서플로 및 케라스 관련 라이브러리'''
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from keras.layers import BatchNormalization, Input, Lambda
from keras.layers import Embedding, Flatten, dot
from keras import regularizers
from keras.losses import mse, binary_crossentropy


In [25]:
'''
무비렌즈 데이터 셋
20,000,263 평점
27,278 영화
138,493 사용자
'''
path = 'C:\\Users\\kt NexR\\PycharmProjects\\Recommendation-algorithms\\Deep Learning_recommendation\\data\\movie-lens'
ratingDF = pd.read_csv(path + '\\ratings.csv', encoding='utf-8')
ratingDF.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [26]:
ratingDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


In [27]:
ratingDF.userId = ratingDF.userId.astype(str).astype(int)
ratingDF.movieId = ratingDF.movieId.astype(str).astype(int)
ratingDF.rating = ratingDF.rating.astype(str).astype(float)
ratingDF.timestamp = ratingDF.timestamp.apply(lambda x: \
                    datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [28]:
ratingDF.describe()

,userId,movieId,rating
count,2.000026e+07,2.000026e+07,2.000026e+07
mean,6.904587e+04,9.041567e+03,3.525529e+00
std,4.003863e+04,1.978948e+04,1.051989e+00
min,1.000000e+00,1.000000e+00,5.000000e-01
25%,3.439500e+04,9.020000e+02,3.000000e+00
50%,6.914100e+04,2.167000e+03,3.500000e+00
75%,1.036370e+05,4.770000e+03,4.000000e+00
max,1.384930e+05,1.312620e+05,5.000000e+00


In [29]:
# 향후 더 빠른 로딩을 위해 데이터 프레임을 피클로 저장
current_path = "C:\\Users\\kt NexR\\PycharmProjects\\Recommendation-algorithms\\Deep Learning_recommendation"
pickle_file = os.path.sep.join(['','datasets'])
file_path = current_path+pickle_file
if not os.path.exists(file_path):
    os.mkdir(file_path)
ratingDF.to_pickle(file_path+'ratingPickle.pkl')

In [30]:
#다시 받아오기 pickle
data = pd.read_pickle(file_path+'ratingPickle.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int32  
 1   movieId    int32  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int32(2), object(1)
memory usage: 457.8+ MB


In [31]:
n_users = ratingDF.userId.unique().shape[0]
n_movies = ratingDF.movieId.unique().shape[0]
n_ratings = len(ratingDF)
avg_ratings_per_user = n_ratings/n_users

print('Number of unique users: ', n_users)
print('Number of unique movies: ', n_movies)
print('Number of total ratings: ', n_ratings)
print('Average number of ratings per user: ', avg_ratings_per_user)

Number of unique users:  138493
Number of unique movies:  26744
Number of total ratings:  20000263
Average number of ratings per user:  144.4135299257002


In [32]:
# 상위 1000개 영화 추출로 데이터 셋 크기 축소
movieIndex = ratingDF.groupby('movieId').count().sort_values(by='rating', ascending = False)[0:1000].index
ratingDFX2 = ratingDF[ratingDF.movieId.isin(movieIndex)]
ratingDFX2.count()

userId       12840344
movieId      12840344
rating       12840344
timestamp    12840344
dtype: int64

In [33]:
# 사용자 1000명 샘플링으로 데이터 셋 크기 축소
userIndex = ratingDFX2.groupby('userId').count().sort_values(by='rating', ascending=False)[0:1000].index
ratingDFX3 = ratingDFX2[ratingDFX2.movieId.isin(userIndex)]
ratingDFX3.count()

userId       90990
movieId      90990
rating       90990
timestamp    90990
dtype: int64

In [11]:
# 영화 ID 다시 인덱싱
movies = ratingDFX3.movieId.unique()
moviesDF = pd.DataFrame(data=movies,columns=['originalMovieId'])
moviesDF['newMovieId'] = moviesDF.index+1
moviesDF.head()

,originalMovieId,newMovieId
0,2657,1
1,586,2
2,741,3
3,2232,4
4,903,5


In [12]:
# 사용자 ID 다시 인덱싱
users = ratingDFX3.userId.unique()
usersDF = pd.DataFrame(data=users, columns=['originalUserId'])
moviesDF['newUserId'] = moviesDF.index+1

In [13]:
ratingDFX3.head()

,userId,movieId,rating,timestamp
391,3,2657,3.0,1999-12-14 12:48:50
446,4,586,4.0,1996-08-24 09:30:38
692,7,2657,2.0,2002-01-16 18:50:19
954,10,2657,4.0,1999-11-25 02:45:34
1005,11,586,4.0,2009-01-02 01:17:07


In [14]:
# 새롭게 병합된 데이터프레임 생성
ratingDFX3 = ratingDFX3.merge(moviesDF,left_on='movieId', \
                              right_on='originalMovieId')
ratingDFX3.drop(labels='originalMovieId', axis=1, inplace=True)
ratingDFX3 = ratingDFX3.merge(usersDF,left_on='userId', \
                              right_on='originalUserId')
ratingDFX3.drop(labels='originalUserId', axis=1, inplace=True)
ratingDFX3.head()

,userId,movieId,rating,timestamp,newMovieId,newUserId
0,3,2657,3.0,1999-12-14 12:48:50,1,1
1,7,2657,2.0,2002-01-16 18:50:19,1,1
2,10,2657,4.0,1999-11-25 02:45:34,1,1
3,72,2657,3.0,2007-12-31 02:05:34,1,1
4,72,586,2.0,2007-12-31 01:58:51,2,2


In [16]:
pickle_file = os.path.sep.join(['', 'datasets', 'ratingReducedPickle'])
ratingDFX3.to_pickle(current_path + pickle_file)
ratingDFX3 = pd.read_pickle(current_path + pickle_file)

In [17]:
ratingDFX3.head()

,userId,movieId,rating,timestamp,newMovieId,newUserId
0,3,2657,3.0,1999-12-14 12:48:50,1,1
1,7,2657,2.0,2002-01-16 18:50:19,1,1
2,10,2657,4.0,1999-11-25 02:45:34,1,1
3,72,2657,3.0,2007-12-31 02:05:34,1,1
4,72,586,2.0,2007-12-31 01:58:51,2,2


In [21]:
n_users = ratingDFX3.userId.unique().shape[0]
n_movies = ratingDFX3.movieId.unique().shape[0]
n_ratings = len(ratingDFX3)
avg_ratings_per_user = n_ratings/n_users

print('Number of unique users: ', n_users)
print('Number of unique movies: ', n_movies)
print('Number of total ratings: ', n_ratings)
print('Average number of ratings per user: ', avg_ratings_per_user)

Number of unique users:  54751
Number of unique movies:  8
Number of total ratings:  90990
Average number of ratings per user:  1.6618874541104272
